In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re
from IPython.display import HTML, display

In [19]:
def clean_arabic_text(text):
    clean_text = re.sub(r'[^ا-ي\s]', '', text)
    return clean_text

In [38]:
As = re.compile(r'[إأآ]')
Ys = re.compile(r'[يى]')
Hs = re.compile(r'[ةه]')
Tashkel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')

def normalize_arabic(text):
    text = re.sub(As, '', text)
    text = re.sub(Ys, 'ا', text)
    text = re.sub(Hs, 'ي', text)
    text = re.sub(Tashkel, '', text)
    return text


In [3]:
def load_model():
    model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa-ner"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    return ner_pipeline

In [4]:
ner_pipeline = load_model()


2024-09-28 14:02:40.651 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 14:02:41.899 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-28 14:02:41.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 14:02:41.909 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 14:02:42.418 Thread 'Thread-10': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 14:02:42.421 Thread 'Thread-10': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the 

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-msa-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2024-09-28 14:02:58.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-28 14:02:58.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [5]:
def process_ner_results(ner_results):
    entities = {}
    for entity in ner_results:
        entity_type = entity['entity_group']
        entity_text = entity['word']
        if entity_type not in entities:
            entities[entity_type] = set()
        entities[entity_type].add(entity_text)
    return entities

In [6]:
def extract_named_entities(text):
    ner_results = ner_pipeline(text)
    return process_ner_results(ner_results)


In [33]:
def highlight_text_html(text, green_words, red_words):
    highlighted_text = text
    for word in green_words:
        highlighted_text = highlighted_text.replace(word, f'<span style="color: green;">{word}</span>')
    for word in red_words:
        highlighted_text = highlighted_text.replace(word, f'<span style="color: red;">{word}</span>')
    display(HTML(f"<p>{highlighted_text}</p>"))

In [26]:
text='''
حرب أكتوبر أو حرب العاشر من رمضان كما تعرف في مصر أو حرب تشرين التحريرية كما تعرف في سوريا أو حرب يوم الغفران
(بالعبرية: מלחמת יום כיפור، ميلخمت يوم كيبور) كما تعرف في إسرائيل، هي حرب شنتها كل من مصر وسوريا في وقتٍ واحدٍ على إسرائيل عام 1973 وهي رابع الحروب العربية الإسرائيلية
 بعد حرب 1948 (حرب فلسطين) وحرب 1956 (حرب السويس) وحرب 1967 (حرب الستة أيام) بخلاف حرب الاستنزاف (1967-1970) التي لم تكن مواجهات عسكرية مباشرة ومستمرة بين الطرفين ولكن غارات
وعمليات عسكرية متفرقة. وكانت إسرائيل في الحرب الثالثة (حرب 1967) قد احتلت شبه جزيرة سيناء من مصر وهضبة الجولان من سوريا، بالإضافة إلى الضفة الغربية التي كانت تحت الحكم الأردني
وقطاع غزة الخاضع آنذاك لحكم عسكري مصري. بدأت الحرب يوم السبت 6 أكتوبر/ تشرين الأول 1973 م الموافق 10 رمضان 1393 هـ بتنسيق هجومين مفاجئين ومتزامنين على القوات الإسرائيلية؛
 أحدهما للجيش المصري على جبهة سيناء المحتلة وآخر للجيش السوري على جبهة هضبة الجولان المحتلة. وقد ساهمت في الحرب بعض الدول العربية سواء بالدعم العسكري أو الاقتصادي.
في 28 سبتمبر/أيلول 1970 توفي الرئيس جمال عبد الناصر، وانتخب نائبه أنور السادات رئيساً لمصر في 15 أكتوبر/تشرين الأول 1970. عقد السادات النية على دخول الحرب وأعلن ذلك في عدة مناسبات منها
إعلانه في 22 يونيو/حزيران 1971 أن عام 1971 هو عام الحسم، وكلامه أمام المجلس الأعلى للقوات المسلحة في 24 أكتوبر/تشرين الأول 1972 الذي أوضح فيه وجوب تجهيز القوات المسلحة لدخول الحرب.
في عام 1973 قرر الرئيسان المصري أنور السادات والسوري حافظ الأسد اللجوء إلى الحرب لاسترداد الأرض التي خسرتها الدولتان في حرب حرب 1967، فقرر مجلس اتحاد الجمهوريات العربية في 10 يناير/كانون
الثاني 1973 تعيين الفريق أول أحمد إسماعيل علي قائداً عاما للقوات الاتحادية، وخلال يومي 22 و23 أغسطس/آب 1973 اجتمع القادة العسكريون السوريون برئاسة مصطفى طلاس وزير الدفاع مع القادة العسكريين المصريين
 برئاسة أحمد إسماعيل علي في الإسكندرية سراً ليشكلوا معاً المجلس الأعلى للقوات المسلحة المصرية والسورية المكون من 13 قائد، وذلك للبت في الموضوعات العسكرية المشتركة والاتفاق النهائي على موعد الحرب،
 واتفق في هذا الاجتماع على بدء الحرب في أكتوبر/تشرين الأول 1973، وخلال اجتماع السادات مع الأسد في دمشق يومي 28 و29 أغسطس/آب اتفقا على أن يكون يوم 6 أكتوبر/تشرين الأول 1973 هو يوم بدء الحرب.

'''



In [39]:
nor_text=normalize_arabic(text)

In [40]:
cleaned_text=clean_arabic_text(nor_text)

In [41]:
NE=extract_named_entities(cleaned_text)

In [42]:
green_words = list(NE['LOC'])
red_words = list(NE['PERS'])

highlight_text_html(text, green_words, red_words)